<a href="https://colab.research.google.com/github/whispermy/novatus_academia/blob/main/pbl/pbl_lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/pbl/trainingData2.csv")
data.head()

,INPUT,OUTPUT,종류
0,A는 B,A = B,선언
1,A는 B로 설정,A = B,선언
2,A는 B로 초기화,A = B,선언
3,A를 B로,A = B,선언
4,A에다가 입력받아줘,A = input(),입력


In [3]:
#형태소 분석기 사용하기 
!pip install konlpy

     |████████████████████████████████| 19.4 MB 45.0 MB/s 
     |████████████████████████████████| 86 kB 4.7 MB/s 
     |████████████████████████████████| 448 kB 47.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [4]:
from konlpy.tag import Kkma, Komoran, Okt, Mecab

okt = Okt()
txt = data['INPUT'][199]
print(okt.pos(txt,norm=True, stem=False, join=False))

[('만약', 'Noun'), ('A', 'Alpha'), ('가', 'Verb'), ('NonE', 'Alpha'), ('값', 'Noun'), ('이', 'Josa'), ('아닐', 'Adjective'), ('때', 'Noun')]


In [5]:
list_input = []

for txt in data['INPUT']:
  tokens = okt.pos(txt,norm=True, stem=False, join=False)
  s = []
  for token in tokens:
    s.append(token[0])
  list_input.append(s)

list_input[:10]


[['A', '는', 'B'],
 ['A', '는', 'B', '로', '설정'],
 ['A', '는', 'B', '로', '초기', '화'],
 ['A', '를', 'B', '로'],
 ['A', '에다가', '입력', '받아줘'],
 ['A', '에', '입력', '받아'],
 ['A', '에', '문자열', 'B', '로', '초기', '화'],
 ['A', '를', '문자열', 'B', '로'],
 ['문자열', 'A', '를', '출력', '해'],
 ['A', '를', '출력', '해']]

In [6]:
#word2vec
from gensim.models import Word2Vec
dimension_word = 50
# train word2vec model
model_input = Word2Vec(list_input, size=dimension_word, window=2, workers=8, min_count=1)
# summarize vocabulary size in model
words = list(model_input.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 150


In [7]:
input_vector = model_input.wv
input_vector['문자열']

array([ 0.00681457,  0.00917972, -0.00020886, -0.00935412, -0.00302224,
       -0.00591075, -0.00570885,  0.00386204, -0.00026905,  0.00087248,
       -0.00568048,  0.00197433, -0.00766818, -0.00916034, -0.00983635,
        0.00131893, -0.00783752,  0.00837995,  0.00357566,  0.00365959,
        0.0093515 ,  0.0016174 ,  0.00312651,  0.0004466 , -0.000528  ,
       -0.01069359,  0.00861054, -0.00792171,  0.0051148 ,  0.00150896,
        0.00715382, -0.00914776, -0.00144025,  0.005483  , -0.00539093,
       -0.00331204,  0.01104489, -0.00644086, -0.00263426,  0.005149  ,
        0.00761539,  0.00297338,  0.0029197 , -0.00375791, -0.00641774,
        0.00374203, -0.00907169,  0.01076811,  0.00312032, -0.00164318],
      dtype=float32)

In [8]:
from keras.preprocessing.text import Tokenizer

tokenizer_input = Tokenizer(lower=False)
tokenizer_input.fit_on_texts(list_input)
encoded_input = tokenizer_input.texts_to_sequences(list_input)
encoded_input[:10]

[[1, 41, 2],
 [1, 41, 2, 15, 83],
 [1, 41, 2, 15, 27, 52],
 [1, 10, 2, 15],
 [1, 116, 84, 117],
 [1, 53, 84, 118],
 [1, 53, 61, 2, 15, 27, 52],
 [1, 10, 61, 2, 15],
 [61, 1, 10, 47, 22],
 [1, 10, 47, 22]]

In [9]:
#input 가중치 행렬
import numpy as np

input_vector_dic = {}
for key in input_vector.wv.vocab.keys():
    input_vector_dic[key] = input_vector[key]
input_vector_dic['A']

# total vocabulary size plus 0 for unknown words
vocab_size=len(tokenizer_input.word_index)+1
input_embedding_vectors = np.zeros((vocab_size, dimension_word))
for word, i in tokenizer_input.word_index.items():
    input_embedding_vectors[i] = input_vector_dic.get(word)

input_embedding_vectors[np.isnan(input_embedding_vectors)] = 0

#정규화 
input_embedding_vectors = input_embedding_vectors / input_embedding_vectors.max(axis=0)

input_embedding_vectors.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


(151, 50)

In [10]:
list_output_in = []
list_output_target = []

########################################################
######## decoder 의 input 으로 쓸 데이터 만들기 ########
########################################################
#(decoder의 데이터는 띄어쓰기 단위로 자르기)
for txt in data['OUTPUT']:
  tokens = txt.split()
  s = ['<sos>']
  t = []
  for token in tokens:
    s.append(token)
    t.append(token)
  list_output_in.append(s)
  t.append('<eos>')
  list_output_target.append(t)
  
print(list_output_in[:10])
print(list_output_target[:10])

[['<sos>', 'A', '=', 'B'], ['<sos>', 'A', '=', 'B'], ['<sos>', 'A', '=', 'B'], ['<sos>', 'A', '=', 'B'], ['<sos>', 'A', '=', 'input()'], ['<sos>', 'A', '=', 'input()'], ['<sos>', 'A', '=', 'B'], ['<sos>', 'A', '=', 'B'], ['<sos>', 'print("', 'A', '")'], ['<sos>', 'print(', 'A', ')']]
[['A', '=', 'B', '<eos>'], ['A', '=', 'B', '<eos>'], ['A', '=', 'B', '<eos>'], ['A', '=', 'B', '<eos>'], ['A', '=', 'input()', '<eos>'], ['A', '=', 'input()', '<eos>'], ['A', '=', 'B', '<eos>'], ['A', '=', 'B', '<eos>'], ['print("', 'A', '")', '<eos>'], ['print(', 'A', ')', '<eos>']]


In [11]:
# list_output_target = []

########################################################
######## decoder 의 output 으로 쓸 데이터 만들기 ########
########################################################
#(decoder의 데이터는 띄어쓰기 단위로 자르기)

In [12]:
tokenizer_output = Tokenizer(lower=False)
tokenizer_output.fit_on_texts(list_output_in)
tokenizer_output.fit_on_texts(list_output_target)

encoded_output_in = tokenizer_output.texts_to_sequences(list_output_in)
print(encoded_output_in[:10])

encoded_output_target = tokenizer_output.texts_to_sequences(list_output_target)
print(encoded_output_target[:10])

[[4, 1, 17, 2], [4, 1, 17, 2], [4, 1, 17, 2], [4, 1, 17, 2], [4, 1, 17, 35], [4, 1, 17, 35], [4, 1, 17, 2], [4, 1, 17, 2], [4, 36, 1, 37], [4, 31, 1, 15]]
[[1, 17, 2, 5], [1, 17, 2, 5], [1, 17, 2, 5], [1, 17, 2, 5], [1, 17, 35, 5], [1, 17, 35, 5], [1, 17, 2, 5], [1, 17, 2, 5], [36, 1, 37, 5], [31, 1, 15, 5]]


In [13]:
#word2vec
from gensim.models import Word2Vec

# train word2vec model
model_output = Word2Vec(list_output_in, size=dimension_word, window=2, workers=8, min_count=1)
# summarize vocabulary size in model
words = list(model_output.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 43


In [14]:
output_vector = model_output.wv
output_vector['<sos>']

array([-8.4169966e-04,  3.9573256e-03,  2.8813339e-03,  1.3574884e-02,
        1.8430573e-03,  4.7838977e-03,  1.5418289e-02, -9.6278926e-03,
       -2.1956677e-03, -9.0154266e-04, -1.1671355e-02, -1.8047931e-03,
        1.2494219e-03,  6.2413216e-03,  3.4803294e-03,  2.6611248e-03,
       -1.2754852e-02, -6.9268271e-03, -1.2614439e-02,  8.4791677e-03,
        1.2013347e-02,  1.0298899e-02,  7.8108098e-04, -1.4120115e-02,
       -5.8749504e-03,  1.0336987e-03, -1.7142974e-02,  8.8886339e-03,
       -8.0797635e-03, -7.0810150e-03,  1.0404945e-02,  3.0844051e-03,
       -9.3434211e-03,  1.4683449e-03, -5.4670451e-03,  8.8499160e-03,
       -3.6267838e-03, -7.2699562e-03,  5.0923130e-03, -4.4472530e-03,
       -4.3231989e-03,  5.6496612e-03, -6.7574409e-04,  9.9775465e-03,
        2.6159456e-03,  1.2899832e-02,  2.0091322e-05,  3.9138389e-03,
       -2.1336742e-03,  1.0457357e-02], dtype=float32)

In [15]:
output_vector_dic = {}
for key in model_output.wv.vocab.keys():
    output_vector_dic[key] = output_vector[key]

In [16]:
# total vocabulary size plus 0 for unknown words
vocab_size=len(tokenizer_output.word_index)+1
output_embedding_vectors = np.zeros((vocab_size, dimension_word))
for word, i in tokenizer_output.word_index.items():
    output_embedding_vectors[i] = output_vector_dic.get(word)

output_embedding_vectors[np.isnan(output_embedding_vectors)] = 0

#정규화 
output_embedding_vectors = output_embedding_vectors / output_embedding_vectors.max(axis=0)
output_embedding_vectors.shape

(45, 50)

In [17]:
from keras.preprocessing.sequence import pad_sequences

encoder_max_length = max([len(s) for s in list_input])
encoder_input = pad_sequences(encoded_input, maxlen=encoder_max_length, padding='post')
print(encoder_input.shape)

decoder_max_length_in = max([len(s) for s in list_output_in])
decoder_input = pad_sequences(encoded_output_in, maxlen=decoder_max_length_in, padding='post')
print(decoder_input.shape)

decoder_max_length_target = max([len(s) for s in list_output_target])
decoder_target = pad_sequences(encoded_output_target, maxlen=decoder_max_length_target, padding='post')
print(decoder_target.shape)

(257, 26)
(257, 14)
(257, 14)


In [18]:
np.random.seed(50)
np.random.shuffle(encoder_input)
np.random.seed(50)
np.random.shuffle(decoder_input)
np.random.seed(50)
np.random.shuffle(decoder_target)

print(encoder_input[0])
print(decoder_input[0])
print(decoder_target[0])

[ 1  3  2  4 17 65  5  3  9  6 45  7  8 12  0  0  0  0  0  0  0  0  0  0
  0  0]
[ 4  8  1 20  2 10  6 12  9  3  0  0  0  0]
[ 8  1 20  2 10  6 12  9  3  5  0  0  0  0]


In [19]:
test_ration = int(len(encoder_input) * 0.1)

In [20]:
encoder_input_train = encoder_input[:-test_ration]
encoder_input_test = encoder_input[-test_ration:]

decoder_input_train = decoder_input[:-test_ration]
decoder_input_test = decoder_input[-test_ration:]

decoder_target_train = decoder_target[:-test_ration]
decoder_target_test = decoder_target[-test_ration:]

In [21]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, GRU, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model

#########################################################################
####################encoder 구조를 rnn로 만들어보기######################
#########################################################################

encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(151,dimension_word,weights=[input_embedding_vectors])(encoder_inputs)
encoder_gru = GRU(dimension_word, activation='relu')(enc_emb)
encoder_output = encoder_gru



In [22]:
from tensorflow.keras.layers import TimeDistributed

##################################################################
####################decoder 구조 만들기###########################
##################################################################
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(45,dimension_word, weights=[output_embedding_vectors])(decoder_inputs)
decoder_gru = GRU(dimension_word,return_sequences=True, return_state=True, activation='relu')
decoder_outputs,_ = decoder_gru(dec_emb, initial_state=encoder_output)
decoder_dense = TimeDistributed(Dense(45, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)


In [23]:
##################################################################
######################모형 연결시키기#############################
##################################################################

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [24]:
##################################################################
###################### 모형 컴파일하기 ###########################
##################################################################

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 50)     7550        ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 50)     2250        ['input_2[0][0]']                
                                                                                              

In [51]:
##################################################################
###################### 모형 학습시키기 ###########################
##################################################################

model.fit(x=[encoder_input_train,decoder_input_train], y=decoder_target_train, batch_size=15, epochs=70)


Epoch 1/70
16/16 [==============================] - 3s 158ms/step - loss: 0.1040 - acc: 0.9615
Epoch 2/70
16/16 [==============================] - 3s 160ms/step - loss: 0.0960 - acc: 0.9643
Epoch 3/70
16/16 [==============================] - 3s 161ms/step - loss: 0.0938 - acc: 0.9637
Epoch 4/70
16/16 [==============================] - 3s 162ms/step - loss: 0.0853 - acc: 0.9686
Epoch 5/70
16/16 [==============================] - 3s 160ms/step - loss: 0.0787 - acc: 0.9689
Epoch 6/70
16/16 [==============================] - 3s 162ms/step - loss: 0.0769 - acc: 0.9720
Epoch 7/70
16/16 [==============================] - 3s 158ms/step - loss: 0.0735 - acc: 0.9717
Epoch 8/70
16/16 [==============================] - 2s 153ms/step - loss: 0.0758 - acc: 0.9704
Epoch 9/70
16/16 [==============================] - 3s 159ms/step - loss: 0.0790 - acc: 0.9717
Epoch 10/70
16/16 [==============================] - 3s 158ms/step - loss: 0.0803 - acc: 0.9692
Epoch 11/70
16/16 [==============================

In [52]:
encoder_model_test = Model(encoder_inputs, encoder_output)

In [53]:
decoder_states_inputs = Input(shape=(dimension_word, ))
dec_emb2 = dec_emb
decoder_outputs_test, decoder_states = decoder_gru(dec_emb2, initial_state=decoder_states_inputs)
decoder_outputs_test = decoder_dense(decoder_outputs_test)
decoder_model = Model([decoder_inputs, decoder_states_inputs], [decoder_outputs_test, decoder_states])

In [54]:
input_to_index = tokenizer_input.word_index
index_to_input = tokenizer_input.index_word
target_to_index = tokenizer_output.word_index
index_to_target = tokenizer_output.index_word

In [55]:
def decode_sequence(input_seq):
  states_value = encoder_model_test.predict(input_seq)

  target_seq = np.array(target_to_index['<sos>']).reshape((1,1,1))
  decoded_sentence = ""
  predict_char = ""

  while True:
    output_tokens, state = decoder_model.predict([target_seq, states_value])
    predict_char_index = np.argmax(output_tokens[0][0])
    predict_char = index_to_target[predict_char_index]

    if predict_char == '<eos>':
      break
    
    decoded_sentence = decoded_sentence + ' ' + predict_char
    target_seq[0][0] = predict_char_index
    states_value = [state]

  return decoded_sentence

In [56]:
def seq2input(input_seq):
  sentence = ""
  for i in input_seq:
    if i != 0:
      sentence = sentence + index_to_input[i] + " "
  return sentence

In [57]:
def seq2target(input_seq):
  sentence = ""
  for i in input_seq:
    if i != 0:
      sentence = sentence + index_to_target[i] + " "
  return sentence

In [58]:
for seq_index in [5,4,3,2,1]:
  input_seq = encoder_input_train[seq_index:seq_index+1]
  decoded_sentences = decode_sequence(input_seq)

  print("한국어 : ",seq2input(encoder_input_train[seq_index]))
  print("정답코드 : ", seq2target(decoder_input_train[seq_index]))
  print("예측코드 : ", decoded_sentences, "\n")


한국어 :  만일 A 가 B 보다 작다면 
정답코드 :  <sos> if A < B: 
예측코드 :   if A < B: 

한국어 :  A 가 B 랑 같거나 C 보다 D 가 작 을 때 까지 돌려줘 
정답코드 :  <sos> while A == B or C < D : 
예측코드 :   while A <= B : 

한국어 :  만일 A 가 B 랑 다르고 C 가 D 랑 같으면 
정답코드 :  <sos> if A != B and C == D : 
예측코드 :   if A != B and C == D : 

한국어 :  A 를 출력 해줘 
정답코드 :  <sos> print( A ) 
예측코드 :   print( A ) 

한국어 :  A 가 B 보다 크거나 C 가 D 랑 같고 E 가 F 보다 클 경우 
정답코드 :  <sos> if A > B or C == D and E > F : 
예측코드 :   if A > B or C == D and E > F : 

